In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, recall_score

import warnings
warnings.filterwarnings("ignore")

Заказчик просит, чтобы:

1) доля найденных моделью потенциальных покупателей была максимальной
2) accuracy при этом была не ниже, чем 0.6 (отклонения accuracy на тестовых данных на  ±0.05  допустимы)

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [3]:
data.describe(include='object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


In [4]:
categorical_columns = [col for col in data.columns if data[col].nunique() == 2]
categorical_df = data[categorical_columns].select_dtypes(include='object')

for col in categorical_df.columns:
    most_common = data[col].value_counts().idxmax()
    categorical_df[col] = categorical_df[col].apply(lambda x: 1 if x == most_common else 0)
    
data[categorical_df.columns] = categorical_df[categorical_df.columns]
data

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,1,0,40000,1,Bachelors,Skilled Manual,1,0,0-1 Miles,Europe,42,1
1,24107,1,1,30000,3,Partial College,Clerical,1,1,0-1 Miles,Europe,43,1
2,14177,1,1,80000,5,Partial College,Professional,0,2,2-5 Miles,Europe,60,1
3,24381,0,1,70000,0,Bachelors,Professional,1,1,5-10 Miles,Pacific,41,0
4,25597,0,1,30000,0,Bachelors,Clerical,0,0,0-1 Miles,Europe,36,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,23731,1,1,60000,2,High School,Professional,1,2,2-5 Miles,North America,54,0
996,28672,0,1,70000,4,Graduate Degree,Professional,1,0,2-5 Miles,North America,35,0
997,11809,1,1,60000,2,Bachelors,Skilled Manual,1,0,0-1 Miles,North America,38,0
998,19664,0,1,100000,3,Bachelors,Management,0,3,1-2 Miles,North America,38,1


In [5]:
columns_to_drop = data.select_dtypes(include='object').columns
df = data.drop(columns_to_drop, axis=1)
df

,ID,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,12496,1,0,40000,1,1,0,42,1
1,24107,1,1,30000,3,1,1,43,1
2,14177,1,1,80000,5,0,2,60,1
3,24381,0,1,70000,0,1,1,41,0
4,25597,0,1,30000,0,0,0,36,0
...,...,...,...,...,...,...,...,...,...
995,23731,1,1,60000,2,1,2,54,0
996,28672,0,1,70000,4,1,0,35,0
997,11809,1,1,60000,2,1,0,38,0
998,19664,0,1,100000,3,0,3,38,1


In [6]:
df = df.drop('ID', axis=1)
df

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,1,0,40000,1,1,0,42,1
1,1,1,30000,3,1,1,43,1
2,1,1,80000,5,0,2,60,1
3,0,1,70000,0,1,1,41,0
4,0,1,30000,0,0,0,36,0
...,...,...,...,...,...,...,...,...
995,1,1,60000,2,1,2,54,0
996,0,1,70000,4,1,0,35,0
997,1,1,60000,2,1,0,38,0
998,0,1,100000,3,0,3,38,1


In [7]:
X = df.drop('Purchased Bike', axis = 1)
y = df['Purchased Bike']

In [8]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
mms = MinMaxScaler()
mms.fit(Xtrain)

Xtrain_mms = pd.DataFrame(data=mms.transform(Xtrain), columns=mms.get_feature_names_out())
Xtest_mms = pd.DataFrame(data=mms.transform(Xtest), columns=mms.get_feature_names_out())

Xtrain_mms

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age
0,1.0,1.0,0.2500,0.2,1.0,0.00,0.140625
1,0.0,1.0,0.5000,0.0,0.0,0.75,0.140625
2,0.0,1.0,0.1875,0.0,0.0,0.00,0.171875
3,0.0,1.0,0.6250,0.0,0.0,0.75,0.109375
4,1.0,1.0,0.5000,1.0,1.0,0.75,0.250000
...,...,...,...,...,...,...,...
695,1.0,1.0,0.3750,0.4,1.0,0.50,0.421875
696,0.0,0.0,0.0000,0.4,1.0,0.00,0.406250
697,0.0,1.0,0.1250,0.0,1.0,0.25,0.109375
698,0.0,0.0,0.0000,0.4,0.0,0.25,0.671875


In [10]:
lr = LogisticRegression()
lr.fit(Xtrain_mms, ytrain)

pred_test = lr.predict(Xtest_mms)

accuracy_score(ytest, pred_test)

0.58

In [11]:
pf = PolynomialFeatures(degree = 2)

pf.fit(Xtrain_mms)

Xtrain = pd.DataFrame(data=pf.transform(Xtrain_mms), columns=pf.get_feature_names_out()) 
Xtest = pd.DataFrame(data=pf.transform(Xtest_mms), columns=pf.get_feature_names_out())
Xtest

,1,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Marital Status^2,Marital Status Gender,...,Children^2,Children Home Owner,Children Cars,Children Age,Home Owner^2,Home Owner Cars,Home Owner Age,Cars^2,Cars Age,Age^2
0,1.0,0.0,1.0,0.1875,0.8,1.0,0.50,0.578125,0.0,0.0,...,0.64,0.8,0.40,0.462500,1.0,0.50,0.578125,0.2500,0.289062,0.334229
1,1.0,1.0,1.0,0.3750,0.4,0.0,0.50,0.375000,1.0,1.0,...,0.16,0.0,0.20,0.150000,0.0,0.00,0.000000,0.2500,0.187500,0.140625
2,1.0,1.0,1.0,0.1875,0.8,0.0,0.00,0.078125,1.0,1.0,...,0.64,0.0,0.00,0.062500,0.0,0.00,0.000000,0.0000,0.000000,0.006104
3,1.0,1.0,0.0,0.3125,0.2,1.0,0.00,0.171875,1.0,0.0,...,0.04,0.2,0.00,0.034375,1.0,0.00,0.171875,0.0000,0.000000,0.029541
4,1.0,1.0,1.0,0.3750,1.0,1.0,0.50,0.281250,1.0,1.0,...,1.00,1.0,0.50,0.281250,1.0,0.50,0.281250,0.2500,0.140625,0.079102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.0,1.0,0.0,0.4375,1.0,1.0,0.75,0.234375,1.0,0.0,...,1.00,1.0,0.75,0.234375,1.0,0.75,0.234375,0.5625,0.175781,0.054932
296,1.0,1.0,0.0,0.3125,0.2,1.0,0.25,0.312500,1.0,0.0,...,0.04,0.2,0.05,0.062500,1.0,0.25,0.312500,0.0625,0.078125,0.097656
297,1.0,1.0,1.0,0.6250,0.8,1.0,0.75,0.359375,1.0,1.0,...,0.64,0.8,0.60,0.287500,1.0,0.75,0.359375,0.5625,0.269531,0.129150
298,1.0,1.0,1.0,0.1875,0.2,1.0,0.00,0.281250,1.0,1.0,...,0.04,0.2,0.00,0.056250,1.0,0.00,0.281250,0.0000,0.000000,0.079102


In [12]:
lr = LogisticRegression()

lr.fit(Xtrain, ytrain)

pred_test = lr.predict(Xtest)

accuracy_score(ytest, pred_test)

0.6233333333333333

In [13]:
recall_score(ytest, pred_test)

0.6959459459459459

In [14]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

In [15]:
def transfer_to_classes(probas_pair):
    if thr >= probas_pair[0]:
        return 1
    return 0

In [16]:
RecMax = -1
BestThr = -1
BestAcc = -1

lr = LogisticRegression()
lr.fit(XtrainS, ytrainS)

probability = lr.predict_proba(Xval)

for thr in np.arange(0, 1, 0.01):
    sub_classes = list(map(transfer_to_classes, probability))
    accur = accuracy_score(yval, sub_classes)
    if accur >= 0.6:
        recall = recall_score(yval, sub_classes)
        if recall > RecMax:
            RecMax, BestAcc, BestThr = recall, accur, thr

print(BestThr, RecMax, BestAcc)

0.58 0.8518518518518519 0.6142857142857143


In [17]:
def transfer_to_classes(probas_pair):
    if BestThr >= probas_pair[0]:
        return 1
    return 0

lr = LogisticRegression()
lr.fit(Xtrain, ytrain)

probability = lr.predict_proba(Xtest)
sub_classes = list(map(transfer_to_classes, probability))
recall_score(ytest, sub_classes)

0.8040540540540541